In [1]:
import tensorflow_hub as hub
model = hub.load("model")

In [2]:
def get_embedding(s):
    
    embedding=model([s])
    return embedding.numpy()[0]

In [1]:
from elasticsearch import Elasticsearch
es = Elasticsearch("http://localhost:9200")

In [4]:
data=list(i.replace("\n","") for i in open("news-dump_2020_1.txt",encoding='utf-8'))
i=0
l=len(data)
res=[]
while i<l:
    
    if (len(data[i])!=11) or (data[i].count("/")!=3):
        i+=1
    else:
        date = data[i]
        zz=data[i+1].index("$")
        category, title = data[i+1][:zz],data[i+1][zz+1:]
        i+=2
        article=""
        while i<l and (not((len(data[i])==11) and (data[i].count("/")==3))):
            article+=data[i]+" "
            i+=1
        res.append((date,category,title,article))
    
    #if i>100:break

In [67]:
def get_ids(query_phrase,categories,from_date, to_date, size,threshold):
    queryssss = get_embedding(query_phrase)
    
    script_query = {
    "script_score": {
    "query": {
    "bool":
     {


         "filter":
         [
             {
                 "terms":{
                     "category":categories
                 }

             },
             {
                 "range":{
                     "date":{"gte":from_date,
                     "lte":to_date}
                 }

             }

         ]

     }   

    },
    "script": {
    "source": """

    double a =cosineSimilarity(params.query_vector, 'Doc_vector');
    if (a<0) a=0;
    return a;

    """,
    "params": {"query_vector": queryssss }
    }}}
    
    response = es.search(index="documents",body={'size': size,'query': script_query})
    print(response['hits']['hits'])
    return [int(i['_id']) for i in response['hits']['hits'] if float(i['_score'])>=threshold]

In [70]:
def bool_query(query,categories,from_date,to_date,size,threshold):
    def f(i,j):
        op=[]
        st=[]
        no=False 
        while i<=j:

            if q[i]=='(':
                z=f(i+1,p[i]-1)
                i=p[i]+1
                if no:
                    z=set(iii for iii in range(50000) if iii not in z)
                    #z=not z
                st.append(z)


            elif q[i] in ('AND','OR'):

                if q[i]=='OR':
                    while op:

                        operator=op.pop()

                        a=st.pop(-1)
                        b=st.pop(-1)

                        if operator=='AND':
                            st.append(a.intersection(b))
                            #st.append(a and b)
                        else:
                            st.append(a.union(b))
                            #st.append(a or b)
                else:
                    while op and op[-1]=='AND':
                        operator=op.pop()
                        a=st.pop(-1)
                        b=st.pop(-1)
                        st.append(a.intersection(b))
                        #st.append(a and b)

                op.append(q[i])
                i+=1


            elif q[i]=='NOT':
                no=True 
                i+=1 
            else:
                z=ids[q[i]]
                if no:
                    z=set(iii for iii in range(50000) if iii not in z)
                    #z=not z 
                st.append(z)
                i+=1
        while op:
            operator=op.pop()
            a=st.pop(-1)
            b=st.pop(-1)
            if operator=='AND':
                st.append(a.intersection(b))
                #st.append(a and b)
            else:
                st.append(a.union(b))
                #st.append(a or b)
        return st[-1]

    x=[]
    se=set()
    y=[{'a','c'},{'b','d','f'},{'a','b','c','d','e'},{'a','c','f'},{'c','d'}]

    #q='a AND NOT ( b AND c )'
    
    q=query.replace("("," ( ").replace(")"," ) ").split()
    l=len(q)
    st=[]
    p={}
    ids={}
    for i in range(l):
        if q[i]=='(':
            st.append(i)
        elif q[i]==')':
            p[st.pop(-1)]=i 
        elif q[i] not in ("AND","OR","NOT"):
            if q[i] not in ids:
                ids[q[i]]=set(get_ids(q[i],categories,from_date,to_date,size,threshold))
                
    
    
    return f(0,l-1)
        

In [86]:
categories=["tp-sport"]
from_date="2020-01-01"
to_date="2020-06-01"
for i in bool_query("cricket",categories,from_date,to_date,1,0.1):
    print(res[i-1][0])
    pass




C:\Users\ivnag\AppData\Local\Temp/ipykernel_5388/21438713.py:43: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es.search(index="documents",body={'size': size,'query': script_query})
